In [ ]:
def preprocess(path, FT = None):  # 이미지 전처리
  img = Image.open(path)

  if FT != None:
    img = pre_transform(img)  # 전전처리 적용
    img = preprocess_FT(img, FT)
    processed_img_tensor = after_FT_transform(img)  # 후전처리 적용

  elif FT == None:
    processed_img_tensor = transform(img)

  return processed_img_tensor


FT_type = ['phase_only', 'amplitude_only', 'shuffle_global_amplitude', 'shuffle_patch_amplitude', 'average_amplitude', 'shuffle_patch_phase']

# 푸리에 변환 함수
def preprocess_FT(img: Image.Image, Type = None):
  Type = FT_type[Type]

  img_np = np.array(img).astype(np.float32)

  H, W, C = img_np.shape
  out_channels = []

  for i in range(C): # 채널 반복
    channel = img_np[:, :, i]

    fft = np.fft.fft2(channel)
    amp = np.abs(fft)
    phase = np.angle(fft)

    # --------- 다양한 FT 처리 방식 ---------
    if Type == 'phase_only': # 위상 정보만 남김
      # amplitude = 1, phase만 유지
      new_fft = np.exp(1j * phase)

    elif Type == 'amplitude_only': # 진폭 정보만 남김
      # left_frequency = np.abs(fft_result)
      # ifft_result = left_frequency
      new_fft = amp

    else:
      # 아직 구현 안 한 타입들은 기본 원본 유지
      new_fft = fft

    if Type in ["phase_only"]:
      ifft = np.fft.ifft2(new_fft)
    elif Type == "amplitude_only":
      ifft = np.log(amp + 1e-8)
    else:
      ifft = np.fft.ifft(new_fft)

    # 실수부만
    ifft = np.real(ifft)
    # 채널 저장
    out_channels.append(ifft)

  out_img = np.stack(out_channels, axis=-1)

  # 0~255 스케일링
  out_img = out_img - out_img.min()
  out_img = out_img / (out_img.max() + 1e-8)
  out_img = (out_img * 255).astype(np.uint8)

  return Image.fromarray(out_img)